In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 7.6MB/s 


In [2]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

# IMDB 리뷰 토큰화하기

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f1878da5d10>)

In [4]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [5]:
print('리뷰 개수 :',len(train_df)) # 리뷰 개수 출력

리뷰 개수 : 50000


In [6]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [7]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [8]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
1372,ories,-1369
2267,oe,-2264
370,ne,-367
178,▁by,-175
4171,Now,-4168
4185,Un,-4182
3287,▁80',-3284
3089,known,-3086
2326,▁editing,-2323
2937,▁store,-2934


In [9]:
len(vocab_list)

5000

In [10]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

In [11]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



In [12]:
sp.GetPieceSize()

5000

In [16]:
sp.IdToPiece(1078)

'▁someone'

In [17]:
sp.PieceToId('▁someone')

1078

In [18]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [19]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

In [20]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


In [21]:
print(sp.encode('Hello. My name is ha sang chun', out_type=str))
print(sp.encode('Hello. My name is ha sang chun', out_type=int))

['▁Hell', 'o', '.', '▁My', '▁name', '▁is', '▁ha', '▁s', 'ang', '▁ch', 'un']
[4481, 4928, 4945, 1194, 1324, 43, 110, 8, 484, 135, 146]


# 네이버 영화 리뷰 토큰화하기

In [22]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [23]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f187cf3cdd0>)

In [24]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [26]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 200000


In [27]:
print(naver_df.isnull().values.any())

True


In [28]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [30]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 199992


In [31]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [32]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [33]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


In [34]:
vocab_list.sample(10)

,0,1
1098,려는,-1095
4257,ᅩ,-4254
3930,첫,-3927
1319,개도,-1316
4070,킹,-4067
4964,옄,-4961
1815,▁범죄,-1812
3223,하라,-3220
3558,몰,-3555
4731,쳇,-4728


In [35]:
len(vocab_list)

5000

In [36]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [37]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [38]:
sp.GetPieceSize()

5000

In [39]:
sp.IdToPiece(4)

'영화'

In [40]:
sp.PieceToId('영화')

4

In [41]:
sp.DecodeIds([54, 200, 821, 85])

'진짜 최고의 영화입니다 ᄏᄏ'

In [42]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [43]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]


In [50]:
print(sp.encode('너무너무 재밌었어요 ㅠㅠ', out_type=str))
print(sp.encode('너무너무 재밌었어요 ㅠㅠ', out_type=int))

['▁너무너무', '▁재밌었어요', '▁ᅲᅲ']
[1624, 2186, 366]


In [51]:
print(sp.encode('너무너너무 재밌었어요 ㅠㅠ', out_type=str))
print(sp.encode('너무너너무 재밌었어요 ㅠㅠ', out_type=int))

['▁너무', '너', '너무', '▁재밌었어요', '▁ᅲᅲ']
[23, 3346, 17, 2186, 366]


In [52]:
print(sp.encode('너무 너너무 재밌었어요 ㅠㅠ', out_type=str))
print(sp.encode('너무 너너무 재밌었어요 ㅠㅠ', out_type=int))

['▁너무', '▁너', '너무', '▁재밌었어요', '▁ᅲᅲ']
[23, 1176, 17, 2186, 366]
